In [3]:
# Import necessary libraries
import os
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import optimizers
from keras import layers, models
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from lxml import etree
import matplotlib.pyplot as plt

In [4]:
# Define paths to annotation and image directories
annotations_dir = "C:\\Users\\mazen\\Desktop\\archive\\imagess\\img-annotations"
images_dir = "C:\\Users\\mazen\\Desktop\\archive\\imagess\\image"

# Define classes
classes = ['With Helmet', 'Without Helmet']

In [5]:
# Parse XML annotations
def parse_annotation(annotation_file):
    tree = etree.parse(annotation_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text), int(bbox.find('ymax').text)]
        objects.append(obj_struct)
    return objects



In [6]:
# Load images and annotations
def load_data(annotations_dir, images_dir):
    i=0
    X, y = [], []
    for annotation_file in os.listdir(annotations_dir):
        if i>535:
            return
        if annotation_file.endswith('.xml'):
            annotation_path = os.path.join(annotations_dir, annotation_file)
            objects = parse_annotation(annotation_path)
            image_path = os.path.join(images_dir, annotation_file.replace('.xml', '.png'))
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            X.append(preprocess_input(image))
            labels = [0] * len(classes)
            for obj in objects:
                label_idx = classes.index(obj['name'])
                labels[label_idx] = 1
            y.append(labels)
            i=i+1
    return np.array(X), np.array(y)